# Notebook to update hrefs in particular collections

In [ ]:
import boto3
import json

In [ ]:
AWS_ACCESS_KEY_ID = "[CHANGE ME]"
AWS_SECRET_ACCESS_KEY = "[CHANGE ME]"
AWS_SESSION_TOKEN = "[CHANGE ME]"

In [ ]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
)

The `update_json_href` function takes in a bucket_name, s3_prefix, old_href_substring and new_href_substring

In [ ]:
def update_json_href(
    bucket_name, collection_name, old_href_substring, new_href_substring
):
    """Given the bucket name, s3 prefix,
       update all hrefs in the path bucketname/s3_prefix
       and update all the old_href_substring to be new_href_substring.

    Keyword arguments:
    bucket_name -- the s3 bucket name
    collection_name -- the collection name
    old_href_substring -- the string to replace in href
    new_href_substring -- the new href substring
    """
    s3 = s3_client
    s3_prefix = f"{collection_name}/"

    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_prefix)

    for obj in response.get("Contents", []):
        key = obj["Key"]

        # Check if the object is a JSON file
        if key.endswith(".json"):
            response = s3.get_object(Bucket=bucket_name, Key=key)
            json_data = response["Body"].read().decode("utf-8")

            data = json.loads(json_data)
            for assets_key in data["assets"]:
                # Update href property
                data["assets"][assets_key]["href"] = data["assets"][assets_key][
                    "href"
                ].replace(old_href_substring, new_href_substring)
                # Serialize updated JSON
                updated_json = json.dumps(data)

                # Upload updated JSON file back to S3
                s3.put_object(Bucket=bucket_name, Key=key, Body=updated_json)

            print(f"Updated {key}")

The next cell calls update_json_href to update the hlsl30-ej-reprocessed and hlss30-ej-reprocessed collections from the veda-data-store bucket. Specifically, it updates the href instances of "covid-eo-data" to "veda-data-store"

In [ ]:
update_json_href(
    "veda-data-store", "hlsl30-ej-reprocessed", "covid-eo-data", "veda-data-store"
)

In [ ]:
update_json_href(
    "veda-data-store", "hlss30-ej-reprocessed", "covid-eo-data", "veda-data-store"
)